<a href="https://colab.research.google.com/github/VTNay/MEC557-Project/blob/Zhichuan-MA/MEC557_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projects

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.in2p3.fr%2Fenergy4climate%2Fpublic%2Feducation%2Fmachine_learning_for_climate_and_energy/master?filepath=book%2Fnotebooks%2Fprojects.ipynb)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<div class="alert alert-block alert-warning">
    <b>Schedule</b>
    
- Ask your supervisors for the data if not already provided (it is not included in this repository).
- Quick presentation.
- Final project presentation.
    
</div>

<div class="alert alert-block alert-info">
    <b>One problematic, One dataset, One (or more) method(s)</b>
    
- Quality of the dataset is key.
- Results on a clean notebook.
- Explain which method(s) you used and why.
- If a method fails, explain why.

</div>

## Project: Weather station

<img alt="weather" src="https://github.com/VTNay/MEC557-Project/blob/main/images/map.png?raw=1" width=400>

- Suppose there are 5 weather stations that monitor the weather: Paris, Brest, London, Marseille and Berlin.
- The weather station in Paris breaks down
- Can we use the other stations to infer the weather in Paris

### Data set

<img alt="weather" src="https://github.com/VTNay/MEC557-Project/blob/main/images/annual_temperature.png?raw=1" width=400>

- Surface variables: skt, u10, v10, t2m, d2m, tcc, sp, tp, ssrd, blh
- Temporal resolution: hourly
- Spatial resolution: N/A

### First steps

- Look at the correlations between variables.
- What variable do I want to predict
- What time scale am interested in?
- Start with the easy predictions and move on to harder ones
- Are there events that are more predictable than others?

In [34]:
from pathlib import Path
import pandas as pd
import xarray as xr
from functools import reduce
from matplotlib import pyplot as plt
from sklearn import preprocessing

paris_path = Path('/content/drive/My Drive/PHY557_Project/weather/paris')
brest_path = Path('/content/drive/My Drive/PHY557_Project/weather/brest')
london_path = Path('/content/drive/My Drive/PHY557_Project/weather/london')
marseille_path = Path('/content/drive/My Drive/PHY557_Project/weather/marseille')
berlin_path = Path('/content/drive/My Drive/PHY557_Project/weather/berlin')

file_path = {'t2m': 't2m.nc', 'blh': 'blh.nc', 'd2m': 'd2m.nc', 'skt': 'skt.nc', 'sp': 'sp.nc', 'ssrd': 'ssrd.nc', 'tcc': 'tcc.nc', 'tp': 'tp.nc', 'u10': 'u10.nc', 'v10': 'v10.nc'}
City_path = {'Paris': paris_path, 'Brest': brest_path, 'London': london_path, 'Marseille': marseille_path, 'Berlin': berlin_path}

Weather_stations = {'Paris': [], 'Brest': [], 'London': [], 'Marseille': [], 'Berlin': []}
for i in Weather_stations:
  Weather_stations[i] = {'t2m': [], 'blh': [], 'd2m': [], 'skt': [], 'sp': [], 'ssrd': [], 'tcc': [], 'tp': [], 'u10': [], 'v10': []}

for city in Weather_stations:
  for i in Weather_stations[city]:
    temp = xr.open_dataset(Path(City_path[city], file_path[i]))
    temp = temp.to_dataframe()
    if i == 'd2m' or i == 'blh':
      temp = temp.droplevel([1,2])
    else:
      temp = temp.droplevel([0,1])
    Weather_stations[city][i] = temp
  #merge them into 1 dataframe
  Weather_stations[city] = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), Weather_stations[city].values())
Weather_stations


{'Paris':                             t2m        blh         d2m         skt  \
 time                                                                 
 1980-01-01 00:00:00  273.624573        NaN  272.332703  271.886169   
 1980-01-01 01:00:00  273.590820        NaN  272.302704  271.277069   
 1980-01-01 02:00:00  273.077484        NaN  272.056641  271.015686   
 1980-01-01 03:00:00  272.647949        NaN  271.807556  270.778687   
 1980-01-01 04:00:00  272.320129        NaN  271.532471  269.853119   
 ...                         ...        ...         ...         ...   
 2021-01-01 02:00:00         NaN  67.381950         NaN         NaN   
 2021-01-01 03:00:00         NaN  55.931046         NaN         NaN   
 2021-01-01 04:00:00         NaN  42.157402         NaN         NaN   
 2021-01-01 05:00:00         NaN  37.912605         NaN         NaN   
 2021-01-01 06:00:00         NaN  35.246353         NaN         NaN   
 
                                 sp  ssrd       tcc            tp 

In [35]:
Berlin = pd.DataFrame(Weather_stations['Berlin'])
Brest =  pd.DataFrame(Weather_stations['Brest'])
London = pd.DataFrame(Weather_stations['London'])
Paris = pd.DataFrame(Weather_stations['Paris'])
Marseille = pd.DataFrame(Weather_stations['Marseille'])


In [42]:
Paris = Paris[Paris.index < '2020-01-01 07:00:00']
Paris

,t2m,blh,d2m,skt,sp,ssrd,tcc,tp,u10,v10
time,,,,,,,,,,
1980-01-01 00:00:00,273.624573,NaN,272.332703,271.886169,100071.015625,NaN,1.000000,NaN,1.821228,0.593918
1980-01-01 01:00:00,273.590820,NaN,272.302704,271.277069,100026.843750,NaN,0.986572,NaN,1.522354,0.792664
1980-01-01 02:00:00,273.077484,NaN,272.056641,271.015686,99995.453125,NaN,1.000000,NaN,1.478912,0.811569
1980-01-01 03:00:00,272.647949,NaN,271.807556,270.778687,99981.320312,NaN,0.989990,NaN,1.310043,0.710587
1980-01-01 04:00:00,272.320129,NaN,271.532471,269.853119,99924.750000,NaN,0.916901,NaN,1.426117,0.915924
...,...,...,...,...,...,...,...,...,...,...
2020-01-01 02:00:00,273.403381,56.836708,273.301880,273.251343,101831.046875,0.0,1.000000,0.000017,-0.432419,1.644012
2020-01-01 03:00:00,273.530762,61.645226,273.487396,273.334076,101812.593750,0.0,1.000000,0.000014,0.383301,1.820221
2020-01-01 04:00:00,273.733459,68.935791,273.716492,273.837219,101798.617188,0.0,1.000000,0.000006,0.362213,1.744476


***
## Credit

[//]: # "This notebook is part of [E4C Interdisciplinary Center - Education](https://gitlab.in2p3.fr/energy4climate/public/education)."
Contributors include Bruno Deremble and Alexis Tantet.
Several slides and images are taken from the very good [Scikit-learn course](https://inria.github.io/scikit-learn-mooc/).

<br>

<div style="display: flex; height: 70px">
    
<img alt="Logo LMD" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_lmd.jpg?raw=1" style="display: inline-block"/>

<img alt="Logo IPSL" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_ipsl.png?raw=1" style="display: inline-block"/>

<img alt="Logo E4C" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_e4c_final.png?raw=1" style="display: inline-block"/>

<img alt="Logo EP" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_ep.png?raw=1" style="display: inline-block"/>

<img alt="Logo SU" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_su.png?raw=1" style="display: inline-block"/>

<img alt="Logo ENS" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_ens.jpg?raw=1" style="display: inline-block"/>

<img alt="Logo CNRS" src="https://github.com/VTNay/MEC557-Project/blob/main/images/logos/logo_cnrs.png?raw=1" style="display: inline-block"/>
    
</div>

<hr>

<div style="display: flex">
    <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0; margin-right: 10px" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
    <br>This work is licensed under a &nbsp; <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.
</div>